In [2]:
import pandas as pd

dataset = pd.read_csv('housing.csv')

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [9]:
!pip install -U sagemaker

In [3]:
print(dataset.shape)
dataset[:5]

(506, 13)


,crim,zn,indus,chas,nox,age,rm,dis,rad,tax,ptratio,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,5.33,36.2


In [4]:
# Move 'medv' column to front
dataset = pd.concat([dataset['medv'], dataset.drop(['medv'], axis=1)], axis=1)

In [5]:
from sklearn.model_selection import train_test_split
training_dataset, validation_dataset = train_test_split(dataset, test_size=0.1)

print(training_dataset.shape)
print(validation_dataset.shape)

(455, 13)
(51, 13)


In [6]:
training_dataset.to_csv('training_dataset.csv', index=False, header=False)
validation_dataset.to_csv('validation_dataset.csv', index=False, header=False)

In [7]:
import sagemaker

print(sagemaker.__version__)

sess = sagemaker.Session()
bucket = sess.default_bucket()

prefix = 'boston-housing'
training_data_path = sess.upload_data(path='training_dataset.csv', key_prefix=prefix + '/input/training')
validation_data_path = sess.upload_data(path='validation_dataset.csv', key_prefix=prefix + '/input/validation')

print(training_data_path)
print(validation_data_path)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
2.199.0
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
s3://sagemaker-us-east-1-918870134825/boston-housing/input/training/training_dataset.csv
s3://sagemaker-us-east-1-918870134825/boston-housing/input/validation/validation_dataset.csv


In [42]:
import boto3
from sagemaker import image_uris

region = boto3.Session().region_name    
container = image_uris.retrieve('xgboost', region, version='1.2-1')

print(container)


role = sagemaker.get_execution_role() 

xgb_estimator = sagemaker.estimator.Estimator(container,
                                    role=role,
                                    instance_count=1,
                                    instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    #sagemaker_session=sagemaker_session
                                )

xgb_estimator.set_hyperparameters(
    objective='reg:squarederror',  # Setting the objective for regression
    num_round=100,  # Number of boosting rounds
    max_depth=5  # Maximum tree depth
)

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.2-1
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [43]:
training_data_channel   = sagemaker.TrainingInput(s3_data=training_data_path, content_type='text/csv')
validation_data_channel = sagemaker.TrainingInput(s3_data=validation_data_path, content_type='text/csv')

xgb_data = {'train': training_data_channel, 'validation': validation_data_channel}

In [44]:
xgb_estimator.fit(xgb_data)

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2023-12-12-10-06-34-974


2023-12-12 10:06:35 Starting - Starting the training job......
2023-12-12 10:07:10 Starting - Preparing the instances for training......
2023-12-12 10:08:34 Downloading - Downloading input data......
2023-12-12 10:09:09 Training - Downloading the training image...
2023-12-12 10:10:04 Training - Training image download completed. Training in progress...[2023-12-12 10:10:13.035 ip-10-0-122-39.ec2.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value reg:squarederror to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determi

In [45]:
%%bash -s "$xgb_estimator.output_path"
aws s3 ls --recursive $1

2023-12-12 10:04:19          0 boston-housing/output/sagemaker-xgboost-2023-12-12-10-00-38-436/debug-output/claim.smd
2023-12-12 10:04:19       6276 boston-housing/output/sagemaker-xgboost-2023-12-12-10-00-38-436/debug-output/collections/000000000/worker_0_collections.json
2023-12-12 10:04:19        218 boston-housing/output/sagemaker-xgboost-2023-12-12-10-00-38-436/debug-output/events/000000000000/000000000000_worker_0.tfevents
2023-12-12 10:04:19        224 boston-housing/output/sagemaker-xgboost-2023-12-12-10-00-38-436/debug-output/events/000000000010/000000000010_worker_0.tfevents
2023-12-12 10:04:19        224 boston-housing/output/sagemaker-xgboost-2023-12-12-10-00-38-436/debug-output/events/000000000020/000000000020_worker_0.tfevents
2023-12-12 10:04:19        224 boston-housing/output/sagemaker-xgboost-2023-12-12-10-00-38-436/debug-output/events/000000000030/000000000030_worker_0.tfevents
2023-12-12 10:04:19        224 boston-housing/output/sagemaker-xgboost-2023-12-12-10-00-38

In [46]:
from time import strftime, gmtime
timestamp = strftime('%d-%H-%M-%S', gmtime())

endpoint_name = 'xgb-demo-'+timestamp
print(endpoint_name)

xgb-demo-12-10-10-55


In [47]:
xgb_predictor = xgb_estimator.deploy(endpoint_name=endpoint_name, 
                        initial_instance_count=1, 
                        instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2023-12-12-10-10-57-237
INFO:sagemaker:Creating endpoint-config with name xgb-demo-12-10-10-55
INFO:sagemaker:Creating endpoint with name xgb-demo-12-10-10-55


--------!

In [48]:
test_sample = '0.00632,18.00,2.310,0,0.5380,6.5750,65.20,4.0900,1,296.0,15.30,4.98'

In [49]:
#XGB_predictor.content_type = 'text/csv'
xgb_predictor.serializer = sagemaker.serializers.CSVSerializer()
xgb_predictor.deserializer = sagemaker.deserializers.CSVDeserializer()

response =xgb_predictor.predict(test_sample)
print(response)

[['24.018329620361328']]


In [51]:
xgb_estimatoramples = ['0.00632,18.00,2.310,0,0.5380,6.5750,65.20,4.0900,1,296.0,15.30,4.98',
                '0.02731,0.00,7.070,0,0.4690,6.4210,78.90,4.9671,2,242.0,17.80,9.14']

response = xgb_predictor.predict(test_sample)
print(response)
print(xgb_predictor.endpoint_name)

[['24.018329620361328']]
xgb-demo-12-10-10-55


In [52]:
runtime = boto3.Session().client(service_name='runtime.sagemaker') 

response = runtime.invoke_endpoint(EndpointName=endpoint_name, 
                                  ContentType='text/csv', 
                                  Body=test_sample)

print(response['Body'].read())

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


b'24.018329620361328'


In [53]:
xgb_predictor.delete_endpoint()

INFO:sagemaker:Deleting endpoint configuration with name: xgb-demo-12-10-10-55
INFO:sagemaker:Deleting endpoint with name: xgb-demo-12-10-10-55
